# Homework 3  - Which book would you recomend?

# Pre-considerations

The initial dataset we use in the first cell was created through a **data scraping process**. Using various functions and libraries, described in the other files of this project, a dataset of about 30k lines was created, in which each line represents a book with the respective columns / attributes.

So the previous steps were:

-1.1. Get the list of books

-1.2. Crawl books

-1.3 Parse downloaded pages

The data scraping process ended with the creation of a file called "articles.tsv" which, managed through pandas, provides us with the dataset on which to work to create the required search engines.

In [1]:
from bs4 import BeautifulSoup
import string
import requests
from selenium import webdriver
import spacy
import os
import pandas as pd
import re
from itertools import islice
import numpy as np
from nltk.tokenize import word_tokenize 
import nltk 
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
import csv
nltk.download('stopwords')
nltk.download('punkt')
import heapq 
import csv
import json

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import invIndex_and_utils
import launch

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


-----------------------------------------------------------------

### Let's visualize the dataset!

In [3]:
df = pd.read_csv('articles.tsv', sep = "\t")

In [5]:
df.head(2)

,bookTitle,bookSeries,bookAuthors,ratingValue,ratingCount,reviewCount,Plot,NumberofPages,Published,characters,Settings,URL
0,The Hunger Games,NaN,Suzanne Collins,4.33,"6,416,751","172,695","[ruins, place, known, North, America, lies, na...",374,September14th2008byScholasticPress,"['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...","District 12, Panem Capitol, Panem Panem",https://www.goodreads.com/en/book/show/2767052...
1,Harry Potter and the Order of the Phoenix,NaN,J.K. Rowling,4.50,"2,529,250","42,835","[door, end, silent, corridor, haunting, Harry,...",870,September2004byScholasticInc.,"['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",Hogwarts School of Witchcraft and Wizardry Lon...,https://www.goodreads.com/en/book/show/2.Harry...


In [6]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
bookTitle,27848,26777,Broken,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bookSeries,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bookAuthors,27848,13503,Stephen King,79,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ratingValue,27848,NaN,NaN,NaN,4.03815,0.336174,0,3.85,4.04,4.23,5
ratingCount,27848,16406,2,157,NaN,NaN,NaN,NaN,NaN,NaN,NaN
reviewCount,27848,5529,1,460,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Plot,27848,27066,[],616,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NumberofPages,27848,NaN,NaN,NaN,350.43,279.367,0,230,320,408,14777
Published,27708,23299,October2n,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN
characters,27848,8736,[],18321,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As we can see, we lost 2152 books because of errors in the scraping process.

So our dataset will contain 27848 books, intead of 30000

--------------------------------------------------------------

# 2. Search Engine

**Pre-processing phase**

In this phase we pre-process the data, in particular all the plots, thanks to  the NLKT Python's library



We create a vocabulary taking all the words that we find in the plots of our books. Thanks to the fuction create_vocabulary, not only we find words, but we map each one with a number.

pre-process the data, in particular all the plots, thanks to the NLKT Python's library

In [4]:
#descrip = df['Plot'].tolist()
# this is a dictionary in this format: {word1:0 , word2:1 , ...}
#d = invIndex_and_utils.create_vocabulary(descrip)


#Let's import our vocabulary for time-processing reason. It was create by the function below.
with open('dictionary.json') as json_file: 
    d = json.load(json_file) 
    
#Let's take the d.values and we have a vocabulary of word made by numbers
words_as_number = d.values()


#Let's substitute the terms of the plots with the correspective numbers of dictionary that map the words
plot_list = df['Plot'].tolist()
plot_list_ = invIndex_and_utils.take_plots_pp(plot_list)
plot_list_pp = invIndex_and_utils.maps_plot_by_number(plot_list_ , d)

### 2.1.1) Create your index!

Let's create the inverted index that will be very important in the development of search engines

In [5]:
#inv_index = invIndex_and_utils.inverted_index(plot_list_pp, words_as_number)

#Inverted index for the fist search engine has been saved as .json file and imported now that it's required
with open('inv_index1.json') as json_file: 
    inv_index = json.load(json_file) 


### 2.1.2) Execute the query

Now it's to time to launch the search engine!

**final_list** : it's a list that contains the books' indices, among the books that satisfy the **Conjunctive query**

The result will be a list of books that contains in their plots all the words of the query.

In [6]:
print("Hello, search your list of book:")
query = list(map(str, input().split()))

#Let's execute the query!
final_list = launch.search_engine1(query , d , inv_index)

df2 = df[["bookTitle","Plot","URL"]]
df3 = df2.loc[[x for x in final_list]]
df3

Hello, search your list of book:
young secret


,bookTitle,Plot,URL
43.0,The Secret Garden,"[delightful, enduring, classics, children, lit...",https://www.goodreads.com/en/book/show/2998.Th...
131.0,Fight Club,"[Chuck, Palahniuk, showed, generation, visiona...",https://www.goodreads.com/en/book/show/3623612...
193.0,Fifty Shades of Grey,"[literature, student, Anastasia, Steele, goes,...",https://www.goodreads.com/en/book/show/1081885...
245.0,The Wind-Up Bird Chronicle,"[Japan, highly, regarded, novelist, vaults, ra...",https://www.goodreads.com/en/book/show/11275.T...
266.0,The Diary of a Young Girl,"[Discovered, attic, spent, years, life, Anne, ...",https://www.goodreads.com/en/book/show/48855.T...
...,...,...,...
27417.0,The Legend of the Firefish,"[ , timeless, tale, pursuit, faith, honor, Pac...",https://www.goodreads.com/en/book/show/1311496...
27539.0,Blood Oath,"[Zach, Barrows, ambitious, young, White, House...",https://www.goodreads.com/en/book/show/6662410...
27611.0,Fortune is a Woman,"[met, aftermath, San, Francisco, devastating, ...",https://www.goodreads.com/en/book/show/355715....
27818.0,The Secret in the Old Attic,"[Yellowed, musty, pages, musty, old, attic, bo...",https://www.goodreads.com/en/book/show/48562.T...


-----------------------------------------------------------------

# 2.2) Conjunctive query & Ranking score

**Let's rank the books!**

For the second search engine, given a query, we want to get the top-k documents related to the query. In particular:

- find all the documents that contains all the words in the query, thanks to the first search engine and then sort them by their similarity with the query.


- the score useful to the ranking of the books will be computed by **cosine similarity** between the query and the document(book), considering the **tfidf** of each document related to the terms' query.


In [6]:
#Let's import N (number of documents that cointains i-th word) and the inverted index with tfidf scores. 
with open('N_list.json') as json_file: 
    N_list = json.load(json_file)

with open('inv_index_tfidf.json') as json_file: 
    inv_index_tfidf = json.load(json_file)


In [8]:
print("Hello, search your list of book:")
query = list(map(str, input().split()))

#Let's execute the query!
top_3 = launch.search_engine2(query, d , inv_index, inv_index_tfidf)

books_indexes = [item[0] for item in top_3]
df_n = df[["bookTitle","Plot","URL"]]
df3_n = df_n.loc[[x for x in books_indexes]]
scores_f = [item[1] for item in top_3]
df3_n['Score'] = scores_f 
df3_n

Hello, search your list of book:
young secret


,bookTitle,Plot,URL,Score
131.0,Fight Club,"[Chuck, Palahniuk, showed, generation, visiona...",https://www.goodreads.com/en/book/show/3623612...,0.999999
193.0,Fifty Shades of Grey,"[literature, student, Anastasia, Steele, goes,...",https://www.goodreads.com/en/book/show/1081885...,0.999999
245.0,The Wind-Up Bird Chronicle,"[Japan, highly, regarded, novelist, vaults, ra...",https://www.goodreads.com/en/book/show/11275.T...,0.999999


---------------------------------------------------------------------

# 3. Define a new score!

With this new search engine let's try to build a particular score! 

We start from the hypothesis that books with few characters often turn out to be more reflective, introspective and with less dense plots, while those with many characters could be composed of intertwining and varied stories on several fronts, telling different things.

So let's try to ask the user which of the two types of book he prefers to search, and based on the choice we assign a score to the books.

### Step 1:
We search for books using the conjunctive query of the first search engine.

### Step 2 :
For each book found, according to the number of characters (which fall within 4 ranges) we assign the book a score from 1 to 4

### step 3:
We multiply the score of step 2 by the value of the book's ranking, obtaining the final score

**Launch this cell to try this particular search engine!**

In [10]:
print("Hello, search your list of book:")
query = list(map(str, input().split()))
print("\n now you have two options \n \
1) Do you prefer more introspective books, with few characters being analyzed in a more intimate way? -> press 1 \n \
2) Do you prefer books in which there is a lot of possibility to vary among the \
multitude of shades  of the many characters? -> press 2 \n")

x = input("insert 1 o 2:")
x = int(x)

print("Also, we took the liberty of suggesting the best-rated books, let's hope it's not a probelam, happy reading")


final_list2 = launch.search_engine1(query , d , inv_index)

#Let's build a dataset with the book of the first search engine
dataf = df[["bookTitle","Plot","URL","characters","ratingValue"]]
dataframe = dataf.loc[[x for x in final_list2]]

rating_list = dataframe['ratingValue'].tolist()
characters_list = dataframe['characters'].tolist()

# Let's launch our new Search Engine!
top_k_engine3 = launch.search_engine3 (characters_list , rating_list, x , final_list2)


#Visualize the results
books_indexes2 = [item[0] for item in top_k_engine3]
final_df = dataframe[["bookTitle","Plot","URL","characters","ratingValue"]]
final_df = final_df.loc[[x for x in books_indexes2]]
scores_f2 = [item[1] for item in top_k_engine3]
final_df['Score'] = scores_f2
final_df

Hello, search your list of book:
young secret

 now you have two options 
 1) Do you prefer more introspective books, with few characters being analyzed in a more intimate way? -> press 1 
 2) Do you prefer books in which there is a lot of possibility to vary among the multitude of shades  of the many characters? -> press 2 

insert 1 o 2:2
Also, we took the liberty of suggesting the best-rated books, let's hope it's not a probelam, happy reading


,bookTitle,Plot,URL,characters,ratingValue,Score
8397.0,Assassin's Fate,"[years, ago, epic, fantasy, novel, featuring, ...",https://www.goodreads.com/en/book/show/3068801...,"['The Fool', 'FitzChivalry Farseer', 'Chade Fa...",4.64,18.56
14309.0,Archangel's Enigma,"[Fiction, Vixen, Naasir, feral, powerful, grou...",https://www.goodreads.com/en/book/show/1740984...,"['Elena Deveraux', 'Jessamy (Angel)', 'Raphael...",4.43,17.72
3238.0,Rising Storm,"[Fireheart, traitorous, enemy, Tigerclaw, vanq...",https://www.goodreads.com/en/book/show/295086....,"['Graystripe', 'Fireheart', 'Sandstorm', 'Blue...",4.40,17.60
